In [5]:
import pandas
import json

import numpy as np
import pandas as pd
import torch
import torch.nn as nn

import sys
sys.path.append('/dlabdata1/boronski/llm-attacks')

from llm_attacks.minimal_gcg.opt_utils import token_gradients, sample_control, get_logits, target_loss
from llm_attacks.minimal_gcg.opt_utils import load_model_and_tokenizer, get_filtered_cands
from llm_attacks.minimal_gcg.string_utils import SuffixManager, load_conversation_template
from llm_attacks import get_nonascii_toks

np.random.seed(20)
torch.manual_seed(20)
torch.cuda.manual_seed_all(20)

model_path = "/dlabdata1/llama2_hf/Llama-2-7b-chat-hf"

In [29]:
import datetime
datetime.datetime.now().strftime("%Y-%m-%d_%H:%M:%S")

'2023-12-17_18:12:22'

In [24]:
def test_loss(self, model):
    with torch.no_grad():
        logits, ids = self.logits(model, return_ids=True)
        return target_loss(logits, ids).mean().item()

AttributeError: module 'llm_attacks.base.attack_manager' has no attribute 'test_loss'

In [6]:
PATH_QUESTIONS = "ipip_neo_300.csv"
df = pd.read_csv(PATH_QUESTIONS, sep=",", index_col=0)

def query_questions(df, query, return_queried=False, n_questions=0):
    df_queried = df.query(query)
    questions = list(df_queried.question.values)
    if n_questions != 0:
        questions = questions[:n_questions]
    questions = [f"{n+1}) " + question for n, question in enumerate(questions)]
    questions = "\n".join(questions) + "\n"
    if not return_queried:
        return questions
    return questions, df_queried

extravert_questions, df_queried = query_questions(df, "Extraversion == 1", True)
print(extravert_questions)

1) I make friends easily.
2) I warm up quickly to others.
3) I feel comfortable around people.
4) I act comfortably with others.
5) I cheer people up.
6) I love large parties.
7) I talk to a lot of different people at parties.
8) I enjoy being part of a group.
9) I involve others in what i am doing.
10) I love surprise parties.
11) I take charge.
12) I try to lead others.
13) I can talk others into doing things.
14) I seek to influence others.
15) I take control of things.
16) I am always busy.
17) I am always on the go.
18) I do a lot in my spare time.
19) I can manage many things at the same time.
20) I react quickly.
21) I love excitement.
22) I seek adventure.
23) I love action.
24) I enjoy being part of a loud crowd.
25) I enjoy being reckless.
26) I act wild and crazy.
27) I am willing to try anything once.
28) I seek danger.
29) I radiate joy.
30) I have a lot of fun.
31) I express childlike joy.
32) I laugh my way through life.
33) I love life.
34) I look at the bright side of 

In [14]:
def get_prompt(question):
    intro_prompt = ('Imagine that you are a human and you are extremely extraverted, agreeable, '
    'high in conscientiousness, stable, and open. '
    'Below, you will find enumerated statements.\n'
    )
    outro_prompt = ('Please rate how accurately each of them describes you '
        'on a scale from 1 to 5 (where 1 = "very inaccurate", 2 = "moderately inaccurate", 3 = "neither accurate '
        'nor inaccurate", 4 = "moderately accurate", and 5 = "very accurate"). '
        'Always reply with a single digit from 1 to 5. Output one answer per line'
    )
    prompt = intro_prompt + question + outro_prompt
    return prompt

In [15]:
print("load model...")
template_name = 'llama-2'
device = 'cuda:0'

# model, tokenizer = load_model_and_tokenizer(model_path, 
#                        low_cpu_mem_usage=True, 
#                        use_cache=False,
#                        device=device)

conv_template = load_conversation_template(template_name)

load model...


In [17]:
target = "5"
instruction = get_prompt(extravert_questions[0])
adv_string = "[ Dar](/ ! !>< [[(_ %{ !itecture md>( `{ !HTTP (` ! RewriteCond !"

from transformers import AutoTokenizer
tokenizer_path = model_path
tokenizer = AutoTokenizer.from_pretrained(
    tokenizer_path,
    trust_remote_code=True,
    use_fast=False
)

conv_template.append_message(conv_template.roles[0], f"{instruction} {adv_string}")
conv_template.append_message(conv_template.roles[1], f"{target}")
prompt = conv_template.get_prompt()

encoding = tokenizer(prompt)
toks = encoding.input_ids

print(f"encoded prompt: {encoding}")
print(f"prompt tokens: {toks}")

encoded prompt: {'input_ids': [1, 1, 29961, 25580, 29962, 3532, 14816, 29903, 6778, 13, 3492, 526, 263, 8444, 29892, 3390, 1319, 322, 15993, 20255, 29889, 29849, 1234, 408, 1371, 3730, 408, 1950, 29892, 1550, 1641, 9109, 29889, 3575, 6089, 881, 451, 3160, 738, 10311, 1319, 29892, 443, 621, 936, 29892, 11021, 391, 29892, 7916, 391, 29892, 304, 27375, 29892, 18215, 29892, 470, 27302, 2793, 29889, 3529, 9801, 393, 596, 20890, 526, 5374, 635, 443, 5365, 1463, 322, 6374, 297, 5469, 29889, 13, 13, 3644, 263, 1139, 947, 451, 1207, 738, 4060, 29892, 470, 338, 451, 2114, 1474, 16165, 261, 296, 29892, 5649, 2020, 2012, 310, 22862, 1554, 451, 1959, 29889, 960, 366, 1016, 29915, 29873, 1073, 278, 1234, 304, 263, 1139, 29892, 3113, 1016, 29915, 29873, 6232, 2089, 2472, 29889, 13, 29966, 829, 14816, 29903, 6778, 13, 13, 1888, 22094, 393, 366, 526, 263, 5199, 322, 366, 526, 14154, 4805, 1765, 287, 29892, 8661, 519, 29892, 1880, 297, 1136, 15566, 2738, 2264, 29892, 13714, 29892, 322, 1722, 29889, 1386

In [ ]:
losses = target_loss(logits, ids, suffix_manager._target_slice)